# ReACT with human in the loop
A business individual needs to do data analysis. They know the exact things they want to calculate but wont know how to use pandas for example.

In this case a Langchain Agent (powered by a LLM) would be responsible for understanding the user's query, and then running predefined Chains (pandas operations / API calls) with human verification at the required steps. It might have to plan out a task list and manage it for complex usecases.

- LLM = openai
- Langchain Agents
- Langchain Tools need to be defined for math, pandas

Ref: https://python.langchain.com/docs/modules/agents/toolkits/pandas

In [1]:
from langchain.agents import create_pandas_dataframe_agent
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_types import AgentType

In [5]:
%pip install pandas tabulate
from langchain.llms import OpenAI
import pandas as pd

df = pd.read_csv("../src_docs/active cover.csv")

  Using cached tabulate-0.9.0-py3-none-any.whl (35 kB)
Note: you may need to restart the kernel to use updated packages.


In [6]:
agent = create_pandas_dataframe_agent(OpenAI(temperature=0), df, verbose=True)

# agent = create_pandas_dataframe_agent(
#     ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613"),
#     df,
#     verbose=True,
#     agent_type=AgentType.OPENAI_FUNCTIONS,
# )

In [7]:
agent.run("how many rows are there?")



> Entering new  chain...
Thought: I need to find out how many rows are in the dataframe
Action: python_repl_ast
Action Input: df.shape[0]
Observation: 36
Thought: I now know the final answer
Final Answer: There are 36 rows in the dataframe.

> Finished chain.


'There are 36 rows in the dataframe.'

In [8]:
agent.run("describe the data")



> Entering new  chain...
Thought: I should look at the data and see what it is telling me
Action: python_repl_ast
Action Input: print(df.head())
Observation:           Product  Total SOH Total BestSeller SOH Total NonBestseller SOH  \
0           Jeans    265,057              223,082                  41,975   
1          Chinos    116,449               88,367                  28,082   
2  Knitted Jogger     34,498               23,648                  10,850   
3    woven Shorts     58,752               46,237                  12,515   
4           Cargo     21,025               17,729                   3,296   

   Instock%   Warehouse SOH    Ajio SOR SOH  Flipkart SOR SOH  \
0        98%          63,825        139,051            35,250   
1        97%          36,458         50,690            14,314   
2        92%           8,365         20,134               140   
3        93%          34,910         13,647             7,267   
4        94%           6,652          3,251         

'The data is about product sales, including total stock on hand, stock on hand in warehouses, stock on order from suppliers, stock in transit, total work in progress, 3 month average sales, gross sold, achievement percentage, projected sales, and active cover.'

In [9]:
agent.run("do EDA on the data")



> Entering new  chain...
Thought: I should look at the data to get an understanding of what it contains
Action: python_repl_ast
Action Input: df.describe()
Observation:         Intransit   Active Cover
count         36.0     36.000000
mean           0.0      8.219444
std            0.0      9.385759
min            0.0      0.000000
25%            0.0      2.775000
50%            0.0      4.550000
75%            0.0     10.675000
max            0.0     44.700000
Thought: I should look at the data types
Action: python_repl_ast
Action Input: df.dtypes
Observation: Product                     object
 Total SOH                  object
Total BestSeller SOH        object
Total NonBestseller SOH     object
 Instock%                   object
 Warehouse SOH              object
  Ajio SOR SOH              object
 Flipkart SOR SOH           object
 Fashion Factory SOH        object
 Intransit                   int64
  Total WIP                 object
3 month avg sales           object
 Gross sol

'The correlation between the columns in the dataframe is as follows: Total SOH and Total BestSeller SOH have a correlation of 0.448994, Total NonBestseller SOH and Warehouse SOH have a correlation of 0.964974, Ajio SOR SOH and Flipkart SOR SOH have a correlation of 0.016650, and Projected sales and Active Cover have a correlation of -0.029098.'